![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/4bd64c1deb3c4ffb8ca7ff93c1f1497d).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np

In [3]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/dataTest_carListings.zip', index_col=0)

In [4]:
# Visualización datos de entrenamiento
dataTraining.head()

,Price,Year,Mileage,State,Make,Model
0,34995,2017,9913,FL,Jeep,Wrangler
1,37895,2015,20578,OH,Chevrolet,Tahoe4WD
2,18430,2012,83716,TX,BMW,X5AWD
3,24681,2014,28729,OH,Cadillac,SRXLuxury
4,26998,2013,64032,CO,Jeep,Wrangler


In [5]:
# Visualización datos de test
dataTesting.head()

,Year,Mileage,State,Make,Model
ID,,,,,
0,2014,31909,MD,Nissan,MuranoAWD
1,2017,5362,FL,Jeep,Wrangler
2,2014,50300,OH,Ford,FlexLimited
3,2004,132160,WA,BMW,5
4,2015,25226,MA,Jeep,Grand


In [6]:
import pandas as pd
import numpy as np
import sklearn.neighbors._base
import sys
import matplotlib.pyplot as plt
import keras
import seaborn as  sns
import tensorflow as tf
import seaborn as  sns
import warnings
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

from sklearn import metrics
from sklearn.model_selection import train_test_split
from missingpy import MissForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense,Activation,Layer,Lambda
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from category_encoders import MEstimateEncoder
from sklearn.preprocessing import PolynomialFeatures
from xverse.ensemble import VotingSelector
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import DBSCAN
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split
from missingpy import MissForest
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense,Activation,Layer,Lambda
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.feature_selection import f_regression

# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
# np.random.seed(42)
# y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])
y_variable_name = 'Price'
# target encoding in training data
# Target encoding ##############################################################################################################
print('\n -- Target encoding \n')
df1=dataTraining.copy()
for col in df1.select_dtypes(include=['object']).columns:
    for col2 in df1.select_dtypes(include=['int64','float64']).columns:
        df1[col2 + "_" + col + "_encoded"] = df1.groupby([col,])[col2].transform("mean")

# Apply this encoding to the test data
df2=dataTesting.copy()
for col in df2.select_dtypes(include=['object']).columns:
    df2[y_variable_name + "_" + col + "_encoded"] = df1.groupby([col,])[y_variable_name].transform("mean")
    for col2 in df2.select_dtypes(include=['int64','float64']).columns:
        df2[col2 + "_" + col + "_encoded"] = df2.groupby([col,])[col2].transform("mean")


print('\n -- Remove categorical variables \n')
df1 = df1.select_dtypes(exclude=['object'])
df2 = df2.select_dtypes(exclude=['object'])
df2 = df2[[col for col in df2.columns if col in df1.columns]]

print('\n -- Generate train and test data \n')
from sklearn.model_selection import train_test_split
y = df1[y_variable_name]
df1=df1.drop([y_variable_name],axis=1)
df1 = df1[[col for col in df1.columns if col in df2.columns]]
df2 = df2[[col for col in df2.columns if col in df1.columns]]
XTotal = df1.to_numpy()
yTotal = y.to_numpy()
XTrain, XTest,yTrain, yTest = train_test_split(XTotal, yTotal, test_size=0.35, random_state=0)
XTest2 = df2.to_numpy()

print('\n -- Set null the univariate outliers \n')
XTrain = pd.DataFrame(XTrain)
for col in XTrain.columns:
    if XTrain[col].dtype == 'object':
        continue
    else:
        q1 = XTrain[col].quantile(0.25)
        q3 = XTrain[col].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (iqr * 1.5)
        upper_bound = q3 + (iqr * 1.5)
        np.where(XTrain[col] < lower_bound, np.nan, XTrain[col])
        np.where(XTrain[col] > upper_bound, np.nan, XTrain[col])

print('\n -- Impute the missing values with random forest \n')
from missingpy import MissForest
imputer = MissForest(criterion='squared_error')
XTrainImputado = imputer.fit_transform(XTrain)
XTrainImputado = pd.DataFrame(XTrainImputado)

XTestImputado = imputer.fit_transform(XTest)
XTestImputado = pd.DataFrame(XTestImputado)

XTest2Imputado = imputer.fit_transform(XTest2)
XTest2Imputado = pd.DataFrame(XTest2Imputado)

XTrainImputado=pd.DataFrame(XTrainImputado)
XTestImputado=pd.DataFrame(XTestImputado)
XTest2Imputado=pd.DataFrame(XTest2Imputado)



print('\n -- Generate new features with mathematical operations \n')
columnas = XTrainImputado.columns
for col in columnas:
    XTrainImputado[str(col) + '**2'] = XTrainImputado[col] ** 2
    XTrainImputado[str(col) + '**3'] = XTrainImputado[col] ** 3
    XTestImputado[str(col) + '**2'] = XTestImputado[col] ** 2
    XTestImputado[str(col) + '**3'] = XTestImputado[col] ** 3    
    XTest2Imputado[str(col) + '**2'] = XTest2Imputado[col] ** 2
    XTest2Imputado[str(col) + '**3'] = XTest2Imputado[col] ** 3     
XTrainImputado = XTrainImputado.dropna(axis=1, how='any')
XTestImputado = XTestImputado[[col for col in XTestImputado.columns if col in XTrainImputado.columns]]
XTest2Imputado = XTest2Imputado[[col for col in XTest2Imputado.columns if col in XTrainImputado.columns]]



print('\n -- Selecting features with F filter \n')
limite_F = 4.0
yTrain = yTrain[XTrainImputado.index]
ftest=f_regression(XTrainImputado,yTrain)

plt.plot(ftest[0],'p')
plt.axhline(y=limite_F,ls=':',color='b') 
plt.ylabel('F scores')

XTrainImputadoselection=XTrainImputado[XTrainImputado.columns[ftest[0]>limite_F]] 
XTestImputadoselection=XTestImputado[XTrainImputado.columns[ftest[0]>limite_F]]
XTest2Imputadoselection=XTest2Imputado[XTrainImputado.columns[ftest[0]>limite_F]]




# Generate new features with interactions ##############################################################################################################
poly = PolynomialFeatures(interaction_only=True,include_bias = False)
XTrainImputadoselection = pd.DataFrame(poly.fit_transform(XTrainImputadoselection))
XTestImputadoselection = pd.DataFrame(poly.fit_transform(XTestImputadoselection))
XTest2Imputadoselection = pd.DataFrame(poly.fit_transform(XTest2Imputadoselection))

# Selecting features with F filter ##############################################################################################################
limite_F = 6
yTrain = yTrain[XTrainImputadoselection.index]
ftest=f_regression(XTrainImputadoselection,yTrain)

plt.plot(ftest[0],'p')
plt.axhline(y=limite_F,ls=':',color='b') 
plt.ylabel('F scores')

XTrainImputadoselection=XTrainImputadoselection[XTrainImputadoselection.columns[ftest[0]>limite_F]] 
XTestImputadoselection=XTestImputadoselection[XTestImputadoselection.columns[ftest[0]>limite_F]]
XTest2Imputadoselection=XTest2Imputadoselection[XTest2Imputadoselection.columns[ftest[0]>limite_F]]


# Selecting features with multicollinearity ##############################################################################################################
corr_matrix = XTrainImputadoselection.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
XTrainImputadoselection = XTrainImputadoselection.drop(to_drop, axis=1)
XTestImputadoselection = XTestImputadoselection[[col for col in XTestImputadoselection.columns if col in XTrainImputadoselection.columns]]
XTestImputadoselection = XTestImputadoselection[XTrainImputadoselection.columns]
XTest2Imputadoselection = XTest2Imputadoselection[[col for col in XTest2Imputadoselection.columns if col in XTrainImputadoselection.columns]]
XTest2Imputadoselection = XTest2Imputadoselection[XTrainImputadoselection.columns]



# Train random regressor ######################################################################################################################################
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
reg = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0)
fit = reg.fit(XTrainImputadoselection, yTrain)
yPred = fit.predict(XTestImputadoselection)
yPred2 = fit.predict(XTest2Imputadoselection)
print('\n -- Random Forest Regressor \n')
print('\n -- RMSE: ', mean_squared_error(yTest, yPred) ** 0.5)
# print('\n -- Cross Validation: ', cross_val_score(fit, XTestImputadoselection, yTest, cv=5).mean())

 



 -- Target encoding 


 -- Remove categorical variables 


 -- Generate train and test data 


 -- Set null the univariate outliers 


 -- Impute the missing values with random forest 


 -- Generate new features with mathematical operations 


 -- Selecting features with F filter 



In [25]:
# Guardar predicciones en formato exigido en la competencia de kaggle
yPred2 = fit.predict(XTest2Imputadoselection)
yPred2 = pd.DataFrame(yPred2)
# yPred2.index = yPred2.index + 1
yPred2.columns = ['Price']
yPred2.to_csv('test_submission.csv', index_label='ID')
yPred2.head()

,Price
0,33086.432806
1,48890.354640
2,16288.359943
3,8267.332495
4,37922.332039
